In [26]:
#path to read from ADLS Gen2
file_path = "abfss://users@datalakefinanceanalytics.dfs.core.windows.net/raw/transactions/PS_20174392719_1491204439457_log.csv"

df = spark.read.csv(file_path, header=True, inferSchema=True)
df.show(5)


StatementMeta(mysparkpool, 1, 6, Finished, Available, Finished)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|      

In [27]:
from pyspark.sql.functions import when, col

# Add risk_level column based on amount
df_cleaned = df.withColumn(
    "risk_level",
    when(col("amount") > 10000, "HIGH").otherwise("NORMAL")
)

df_cleaned.select("type", "amount", "risk_level").show(10)


StatementMeta(mysparkpool, 1, 7, Finished, Available, Finished)

+--------+--------+----------+
|    type|  amount|risk_level|
+--------+--------+----------+
| PAYMENT| 9839.64|    NORMAL|
| PAYMENT| 1864.28|    NORMAL|
|TRANSFER|   181.0|    NORMAL|
|CASH_OUT|   181.0|    NORMAL|
| PAYMENT|11668.14|      HIGH|
| PAYMENT| 7817.71|    NORMAL|
| PAYMENT| 7107.77|    NORMAL|
| PAYMENT| 7861.64|    NORMAL|
| PAYMENT| 4024.36|    NORMAL|
|   DEBIT| 5337.77|    NORMAL|
+--------+--------+----------+
only showing top 10 rows



In [28]:
#saving as Spark Delta Table
spark.sql("CREATE DATABASE IF NOT EXISTS financial")


StatementMeta(mysparkpool, 1, 8, Finished, Available, Finished)

DataFrame[]

In [29]:
df_cleaned.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("financial.cleaned_transactions")


StatementMeta(mysparkpool, 1, 9, Finished, Available, Finished)